# Experiment: Adaboost on iris dataset

## 🔍 Quick Summary: AdaBoost (Adaptive Boosting)

**Goal:** Combine many weak learners (e.g., decision stumps) into a strong classifier.

Note: DTs try to solve the entire problem in one shot while adaboost builds a team of simple trees each one improving the last where the final prediciton is a weighted vote.

---

### 🧠 Core Idea:
1. **Train weak learners sequentially**, each focusing more on examples the previous ones got wrong.
2. **Weight the predictions** of each learner based on its accuracy—better learners get more say in the final vote.

---

### 🧮 Algorithm Steps (with Tunable Hyperparameters):
1. **Initialize weights** on all training examples equally.
2. **Repeat for T rounds** *(T = `n_estimators`)*:
   - Train a weak learner *(usually `DecisionTreeClassifier(max_depth=1)` — can tune `base_estimator`, e.g., tree depth, or use a different model)*.
   - Compute the learner’s error on weighted data.
   - Calculate its weight in the final ensemble.
   - Update the weights on training examples *(affected by `learning_rate`)*.
3. **Final prediction** = weighted majority vote of all learners.

---

### 🔧 Key Hyperparameters to Tune:
- `n_estimators` — Number of boosting rounds (T)
- `learning_rate` — Shrinks contribution of each learner (default = 1.0)
- `base_estimator` — The weak learner model (commonly a decision stump)
    - If using a tree: tune `max_depth`, `min_samples_split`, etc.

---

### 🧪 Expected Performance on the Iris Dataset

| Aspect          | Expectation | Notes |
|-----------------|-------------|-------|
| Accuracy        | High (~95–100%) | Clean and well-separated dataset. |
| Overfitting     | Unlikely     | Noisy data is minimal, and the feature space is simple. |
| Training Time   | Very Fast    | Small dataset (150 samples); training takes milliseconds. |
| Inference Time  | Instantaneous | Just a few decision stumps to evaluate. |
| Best Base Learner | Decision stump (`max_depth=1`) | Simple learners work well for AdaBoost on small datasets. |

---

### 🔑 Characteristics:
- **Fast** and **simple**.
- Works well on **tabular** data.
- Can **overfit** on noisy datasets.
- Sensitive to **outliers** because it keeps boosting hard examples.

In [1]:
%reload_ext autoreload
%autoreload 2

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# experiment helper imports
from sklearn.base import clone
from helpers.base_imports import *

## Setup experiment with data and model

In [3]:
exp = Experiment(
    type="c",  # classification
    name="adaboost-iris",
    dataset="iris-20test-shuffled-v1",
)
exp

Loading 'classification-experiments.csv'
Creating experiment: 'adaboost-iris'
No estimator found for 'adaboost-iris'. Create a new one with memory=.cache


Experiment(c, adaboost-iris, iris-20test-shuffled-v1)

We'll start with the regular unpruned DT then check the values of ccp_alpha with a validation curve to see if we can find a good value for pruning.

In [ ]:
RANDOM_SEED = 42
CACHE_DIR = None  # or use a path if you want caching

# Define weak learner (decision stump)
base_stump = DecisionTreeClassifier(
    criterion="entropy",
    splitter="best",
    max_depth=1,  # shallow tree = decision stump
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=RANDOM_SEED,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0,
)

# Build pipeline
steps = [
    # NOTE: Scaling not strictly needed for trees, but good for consistency across models
    ("scaler", StandardScaler()),
    (
        "classifier",
        AdaBoostClassifier(
            estimator=base_stump,
            n_estimators=50,  # heuristic: start with 50, increase to 100-200 if underfitting
            learning_rate=1.0,  # typical range is .01 to 1.0 (heuristic: start with 1, try smaller values with more estimators since lower learning rates need higher estimators to compensate)
            random_state=RANDOM_SEED,
        ),
    ),
]

exp.estimator = Pipeline(
    steps=steps,
    memory=CACHE_DIR,
)

In [6]:
exp.estimator.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()),
  ('classifier',
   AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=1,
                                                       random_state=42),
                      random_state=42))],
 'transform_input': None,
 'verbose': False,
 'scaler': StandardScaler(),
 'classifier': AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                     max_depth=1,
                                                     random_state=42),
                    random_state=42),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'classifier__algorithm': 'deprecated',
 'classifier__estimator__ccp_alpha': 0.0,
 'classifier__estimator__class_weight': None,
 'classifier__estimator__criterion': 'entropy',
 'classifier__estimator__max_depth': 1,
 'classifier__estimator__max_featur

Get dataset by name (eda already done in another notebook and train/test split saved so we will be working with the same data)

In [7]:
notes, X_train, X_test, y_train, y_test, target_names = get_dataset(exp.dataset)
print(notes)
print(target_names)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Dataset: iris-20test-shuffled-v1
X_train shape: (120, 4)
X_test shape: (30, 4)
y_train shape: (120,)
y_test shape: (30,)
Train: 80.00% of total
Test: 20.00% of total
Notes: None
Created by save_dataset() helper at 2024-07-09 12:28:10

  target_names
0       setosa
1   versicolor
2    virginica


((120, 4), (30, 4), (120, 1), (30, 1))

In [8]:
exp.update_param("n_train_samples", X_train.shape[0])
exp.update_param("n_test_samples", X_test.shape[0])
exp.summary_df

,dataset_name,n_train_samples,n_test_samples,mean_accuracy,train_time,query_time,kfolds,confusion_matrix,classification_report
exp_name,,,,,,,,,
adaboost-iris,iris-20test-shuffled-v1,120,30,NaN,NaN,NaN,NaN,NaN,NaN


## Find optimal hyperparameters

What `cv` should we use?

In [11]:
cv_values = [3, 5]
for cv in cv_values:
    print(f"\nRunning GridSearchCV with cv={cv}")
    grid_search = GridSearchCV(
        estimator=exp.estimator,
        param_grid={
            "classifier__n_estimators": [50, 100, 200],
            "classifier__learning_rate": [1.0, 0.5, 0.1],
        },
        cv=cv,
        scoring="accuracy",
        n_jobs=-1,
    )
    grid_search.fit(X_train, y_train)
    print("Best params:", grid_search.best_params_)
    print("Best score:", grid_search.best_score_)


Running GridSearchCV with cv=3


/Users/yarik/vc_projects/ML/machine-learning/.venv-py31013/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yarik/vc_projects/ML/machine-learning/.venv-py31013/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yarik/vc_projects/ML/machine-learning/.venv-py31013/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yarik/vc_projects/ML/machine-learning/.venv-py31013/lib/pyth

Best params: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 100}
Best score: 0.9666666666666667

Running GridSearchCV with cv=5


/Users/yarik/vc_projects/ML/machine-learning/.venv-py31013/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yarik/vc_projects/ML/machine-learning/.venv-py31013/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yarik/vc_projects/ML/machine-learning/.venv-py31013/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yarik/vc_projects/ML/machine-learning/.venv-py31013/lib/pyth

Best params: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 100}
Best score: 0.9583333333333334


/Users/yarik/vc_projects/ML/machine-learning/.venv-py31013/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


`cv=3` does better so we'll stick with that and .1 learning rate with 100 estimators.

## Train

In [12]:
cv = 3
my_params = {
    "classifier__n_estimators": 100,
    "classifier__learning_rate": 0.1,
}
exp.estimator.set_params(**my_params)
exp.estimator.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()),
  ('classifier',
   AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=1,
                                                       random_state=42),
                      learning_rate=0.1, n_estimators=100, random_state=42))],
 'transform_input': None,
 'verbose': False,
 'scaler': StandardScaler(),
 'classifier': AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                     max_depth=1,
                                                     random_state=42),
                    learning_rate=0.1, n_estimators=100, random_state=42),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'classifier__algorithm': 'deprecated',
 'classifier__estimator__ccp_alpha': 0.0,
 'classifier__estimator__class_weight': None,
 'classifier__estimator__criterion': 'entropy',
 

In [14]:
# fit on training data
start_time = pd.Timestamp.now()
exp.estimator.fit(X=X_train, y=y_train)
train_time = pd.Timestamp.now() - start_time

/Users/yarik/vc_projects/ML/machine-learning/.venv-py31013/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
exp.update_param("train_time", train_time)
exp.update_param(
    "mean_accuracy",
    exp.estimator.score(X_test, y_test),
    # add_column=True
)
exp.summary_df

,dataset_name,n_train_samples,n_test_samples,mean_accuracy,train_time,query_time,kfolds,confusion_matrix,classification_report
exp_name,,,,,,,,,
adaboost-iris,iris-20test-shuffled-v1,120,30,0.966667,0 days 00:00:00.073348,NaN,NaN,NaN,NaN


## Take a look at the trained model

In [19]:
exp.estimator.named_steps["classifier"]

AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                    max_depth=1,
                                                    random_state=42),
                   learning_rate=0.1, n_estimators=100, random_state=42)

In [17]:
# convert target names series to list
target_names_list = target_names["target_names"].tolist()
target_names_list

['setosa', 'versicolor', 'virginica']

In [21]:
ada_model = exp.estimator.named_steps["classifier"]
n_trees = len(ada_model.estimators_)
print(f"AdaBoost used {n_trees} trees")

AdaBoost used 100 trees


Take a look at each tree and how it was weighted in the final model.

In [22]:
output_lines = [f"AdaBoost used {n_trees} trees\n"]

for i, stump in enumerate(ada_model.estimators_):
    tree_header = (
        f"\n=== Tree {i + 1} (Weight = {ada_model.estimator_weights_[i]:.3f}) ==="
    )
    tree_text = export_text(stump, feature_names=X_train.columns)

    print(tree_header)
    print(tree_text)

    output_lines.append(tree_header)
    output_lines.append(tree_text + "\n")

# Save to file
with open(f"{RES_DIR}/{exp.name}-adaboost_trees.txt", "w") as f:
    f.write("\n".join(output_lines))


=== Tree 1 (Weight = 0.150) ===
|--- petal width (cm) <= -0.54
|   |--- class: 0
|--- petal width (cm) >  -0.54
|   |--- class: 2


=== Tree 2 (Weight = 0.135) ===
|--- petal length (cm) <= -0.69
|   |--- class: 0
|--- petal length (cm) >  -0.69
|   |--- class: 2


=== Tree 3 (Weight = 0.139) ===
|--- petal length (cm) <= -0.69
|   |--- class: 0
|--- petal length (cm) >  -0.69
|   |--- class: 1


=== Tree 4 (Weight = 0.129) ===
|--- petal length (cm) <= -0.69
|   |--- class: 0
|--- petal length (cm) >  -0.69
|   |--- class: 2


=== Tree 5 (Weight = 0.132) ===
|--- petal width (cm) <= -0.54
|   |--- class: 0
|--- petal width (cm) >  -0.54
|   |--- class: 1


=== Tree 6 (Weight = 0.124) ===
|--- petal width (cm) <= -0.54
|   |--- class: 0
|--- petal width (cm) >  -0.54
|   |--- class: 2


=== Tree 7 (Weight = 0.127) ===
|--- petal length (cm) <= -0.69
|   |--- class: 0
|--- petal length (cm) >  -0.69
|   |--- class: 1


=== Tree 8 (Weight = 0.119) ===
|--- petal width (cm) <= -0.54
|   

In [20]:
for name, importance in zip(
    X_train.columns, exp.estimator.named_steps["classifier"].feature_importances_
):
    print(f"{name}: {importance:.3f}")

sepal length (cm): 0.000
sepal width (cm): 0.000
petal length (cm): 0.515
petal width (cm): 0.485


In [ ]:
exp.save(overwrite_existing=True)